In [1]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

import tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cheol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\cheol\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cheol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cheol\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def stock_prices(ticker):
    """
    ticker is the abbreviated symbol for a stock e.g.AAPL
    this function returns the daily price history of the requested stock as a dataframe
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    response = requests.get(stock_endpoint + ticker + '?apikey=70407133ea11d7284c70bbca4eee2547').json()
    type(response) == dict
    return pd.DataFrame(response['historical'])

def ny_times_articles(keyword):
    url_dict = {}
    index = 0
    for i in range(10):
        response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q="+ keyword +"&fq=news_desk:Business&page="+str(i)+"&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
        try:
            docs = response['response']['docs']
            for item in docs:
                if item['web_url'][8] == 'w': # To remove 'https://bits.blogs.nytimes...' or ''https://dealbook.nytimes...'
                    link = item['web_url']
                    date = link[24:28] + '-' + link[29:31] + '-' + link[32:34]
                    url_dict[index] = tuple((link, date))
                    index += 1
        except KeyError:
            continue
    return url_dict

In [3]:
stocks = stock_prices('AAPL')
url_dict = ny_times_articles('apple')

In [4]:
# use the closing prices of the stock as the daily prices and the differences in each day as the percentage rise or fall
# the difference of stock prices between date1 and date2 will be stored in the row of date1 for ease of comparison

prices = stocks['close']
dates = stocks['date']
difference = stocks['close'].astype('float64').diff()
pct_change = difference / prices

stock = pd.DataFrame({'date': dates, 'price':prices, 'change': difference, 'pct_change': pct_change})
stock

,date,price,change,pct_change
0,2020-12-31,132.690002,NaN,NaN
1,2020-12-30,133.720001,1.029999,0.007703
2,2020-12-29,134.869995,1.149994,0.008527
3,2020-12-28,136.690002,1.820007,0.013315
4,2020-12-24,131.970001,-4.720001,-0.035766
...,...,...,...,...
1254,2016-01-08,24.240000,-0.392500,-0.016192
1255,2016-01-07,24.112499,-0.127501,-0.005288
1256,2016-01-06,25.174999,1.062500,0.042205
1257,2016-01-05,25.677500,0.502501,0.019570


In [5]:
tokenized = tokenizer.tokenize_sentence(url_dict)

In [6]:
merged = tokenizer.merged

In [7]:
tokenized_word = tokenizer.tokenizer_myself(tokenized)

In [8]:
sentence_vals = tokenizer.sentence_calculator(tokenized_word)

In [9]:
article_vals = tokenizer.calculate_vals(sentence_vals)

ZeroDivisionError: division by zero